In [182]:
import numpy as np
import random
import os 
from math import * 

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats=['svg'] 


from subprocess import call

# Methods

def input_state(r, m):

    """
    Returns a list of squeezing parameters
    and a list of input random phases.
    Fills only a half of a number of modes 
    m with equal squeezing parameter r.
    """

    r_ = [r for i in range(m)]
    phi_ = [np.round(np.pi * random.random(), 5) for i in range(m)]

    return r_, phi_


def input_matrix(r, phi, m, n):

    """
    Returns a diagonal input matrix A.
    """

    A = np.zeros((m, m), dtype=np.complex128)

    for i in range(n):
        A[i, i] = -np.exp(1j * phi[i]) * np.tanh(r[i]) / 2

    return A

# Interferometer random matrix


def ps_1(phi, ind, i, m):

    """
    Returns matrix of phase shifter for the first
    random channel of a circuit.
    """

    U = np.eye(m, dtype=np.complex128)

    U[ind[0], ind[0]] = np.exp(1j * phi[i])
    U[ind[1], ind[1]] = 1

    return U


def ps_2(psi, ind, i, m):

    """
    Returns matrix of phase shifter for the second 
    random channel of a circuit.
    """

    U = np.eye(m, dtype=np.complex128)

    U[ind[0], ind[0]] = 1
    U[ind[1], ind[1]] = np.exp(1j * psi[i])

    return U


def bs(eta, i, ind, m):

    """
    Returns matrix of beam splitter for two random 
    channels of a circuit.
    """

    U = np.eye(m, dtype=np.complex128)

    U[ind[0], ind[0]] = np.cos(eta[i])
    U[ind[0], ind[1]] = -np.sin(eta[i])
    U[ind[1], ind[0]] = np.sin(eta[i])
    U[ind[1], ind[1]] = np.cos(eta[i])

    return U

def interferometer(n_bs, m):

    """
    Returns random matrix of interferometer,
    array of random indices for channels' mixing and
    lists of random angles of phase shifters and beam splitters.
    """

    phi = [np.round(2 * np.pi * random.random(), 5) for i in range(n_bs)]
    psi = [np.round(2 * np.pi * random.random(), 5) for i in range(n_bs)]
    eta = [np.round(2 * np.pi * random.random(), 5) for i in range(n_bs)]

    ind = []

    while len(ind) < (n_bs):
        k, j = random.randint(0, m - 1), random.randint(0, m - 1)
        if k != j:
            k_j = sorted([k, j])
            ind.append(k_j)

    U = np.eye(m, dtype=np.complex128)

    for i in range(n_bs):

        U = (
            U
            @ ps_1(phi, ind[i], i, m)
            @ bs(eta, i, ind[i], m)
            @ ps_2(psi, ind[i], i, m)
        )
        
    return U, ind, phi, psi, eta

def M_matrix(U, A):

    """
    Returns the Gaussian multi-mode matrix of a GBS scheme.
    """

    m = len(U)

    M = np.zeros((m, m), dtype=np.complex128)

    for k in range(m):
        for i in range(m):
            for j in range(m):
                M[i, j] += U[k, i] * U[k, j] * A[k, k]

    return M


def average_photon_number(r):
    
    """
    Returns a value of the average photon number 
    in a mode from a list of squeezing parameters
    for each mode.
    """
    
    n_av  = 0
    n = len(r)
    
    for r_i in r:
        n_av += np.sinh(r_i)**2/n
        
    return n_av


# Import/Export

def export_input(r_s, phi_s, A, ind, phi, psi, eta, n_bs, U, M, n, n_mc=10**5, n_cutoff=average_photon_number(r_s), batch_size=10**3, path=path):

    """
    Exports of four files, which contains: 
    1) Gaussian multi-mode matrix of a GBS scheme 
    (GBS_matrix.dat; the odd columns contain a real parts
    of matrix elements, the even columns contain an imaginary 
    parts of matrix elements);
    2) initial state data (initial_state.dat);
    3) parameters of interferometer, which can be used to
    reconstruct the interferometer matrix
    (parameters_of_interferometer.dat); 
    4) the interferometer unitary matrix (matrix_U.dat).
    """

    m = len(M)
    n_ps = int(n_bs*2)
    n = np.count_nonzero(np.array(r_s))

    with open(path + r"/initial_state.dat", "w") as ouf:

        ouf.write(
            "N"
            + "\t"
            + str("r")
            + "\t"
            + str("phi")
            + "\t"
            + str("A_real")
            + "\t"
            + str("A_imag")
            + "\n"
        )

        for k in range(m):
            ouf.write(
                str(k)
                + "\t"
                + str(r_s[k])
                + "\t"
                + str(phi_s[k])
                + "\t"
                + str((A[k, k].real))
                + "\t"
                + str((A[k, k].imag))
            )
            if k < (m + 1):
                ouf.write("\n")

    with open(path + "/parameters_of_interferometer.dat", "w") as ouf:

        ouf.write(
            "# N_modes = "
            + str(m)
            + "\t"
            + str("N_bs = ")
            + str(n_bs)
            + "\t"
            + str("N_ps = ")
            + str(n_ps)
            + "\n"
        )
        ouf.write(
            "[n1, n2]"
            + "\t"
            + str("phi")
            + "\t"
            + str("psi")
            + "\t"
            + str("eta")
            + "\n"
        )

        for z in range(n_bs):
            ouf.write(
                str(ind[z][0])
                + "\t"
                + str(ind[z][1])
                + "\t"
                + str(phi[z])
                + "\t"
                + str(psi[z])
                + "\t"
                + str(eta[z])
                + "\n"
            )
                
    with open(path + "/GBS_matrix.dat", "w") as ouf:
        
        ouf.write(str(m) + "\t" + str(n)+ "\t" + str(r_s[0]) + "\t" + str(n_cutoff)+ "\t" + str(n_mc) + "\t" + str(batch_size) + "\n") 

        for k in range(m):
            for j in range(m):
                ouf.write(str(M[k, j].real) + "\t" + str(M[k, j].imag) + "\t")
            if k < (m + 1):
                ouf.write("\n")
                
    with open(path + r"/matrix_U.dat", "w") as ouf:
        
        for k in range(m):
            for j in range(m):
                ouf.write(str(U[k, j].real) + "\t" + str(U[k, j].imag) + "\t")
            if k < (m + 1):
                ouf.write("\n")
                

    return print("Data were exported to " + path)

def import_input(path, file_name):

    """Imports an Gaussian multi-mode matrix of a GBS scheme. """
   
    data_M = np.genfromtxt(path + file_name, skip_header=1)

    m = len(data_M)
    
    data_ = np.genfromtxt(path + file_name, skip_footer = m )
    
    n, r, n_cutoff, n_mc, batch_size = int(data_[1]), data_[2], int(data_[3]), int(data_[4]), int(data_[5])

    M = np.zeros((m, m), dtype=np.complex128)

    real_part = []
    imaginary_part = []

    for i in range(m):
        for k in range(0, 2 * m, 2):
            real_part.append(data_M[i, k])

    for i in range(m):
        for k in range(1, 2 * m + 1, 2):
            imaginary_part.append(data_M[i, k])

    for i in range(m**2):
        M[i // m, i % m] = real_part[i] + 1j * imaginary_part[i]

    print("Data were imported from " + path + file_name)

    return M, m, n, r, n_cutoff, n_mc, batch_size 

def import_interferometer(path, file_name):

    """Imports an Gaussian multi-mode matrix of a GBS scheme. """
   
    data_U = np.genfromtxt(path + file_name)

    m = len(data_U)
    
    U = np.zeros((m, m), dtype=np.complex128)

    real_part = []
    imaginary_part = []

    for i in range(m):
        for k in range(0, 2 * m, 2):
            real_part.append(data_U[i, k])

    for i in range(m):
        for k in range(1, 2 * m + 1, 2):
            imaginary_part.append(data_U[i, k])

    for i in range(m**2):
        U[i // m, i % m] = real_part[i] + 1j * imaginary_part[i]

    #print("Data were imported from " + path + file_name)

    return U

def uniform_sampling(batch_size, sample_length, n_photons):
    
    """
    Generates samples from the uniform distribution. 
    Using the Mersenne Twister algorithm (module random), 
    it gives pseudo-random uniformly distributed bits 
    (probability 1/2 producing 0 and 1/2 producing 1) on
    sample_length places and it adds iteratively integers 
    to nonzero elements up to n_photons also randomly. 
    Stops when the number of samples is equal to batch_size.     
    """
    
    samples = [] 
    
   
    while len(samples) < batch_size: 

        # sample of uniformly distibuted random bits (Mersenne Twister algorithm)
        
        seed_sample = [random.getrandbits(1) for i in range(n_photons)] + [0]*int(sample_length - n_photons) 
        
        seed_sample = random.sample(seed_sample, len(seed_sample))
        

        add_ph = n_photons - seed_sample.count(1)
        
        uniform_sample = np.copy(seed_sample)
        
        ind_list = [index for (index, item) in enumerate(uniform_sample) if item == 1]
        
        counter = 0 
        
        if ind_list != []:
        
            while counter < add_ph:

                place = random.choice(ind_list)

                uniform_sample[place] += 1

                counter += 1
                    
            samples.append(uniform_sample)

    return np.array(samples)

def device_initialization(m, n, r, n_bs=m**2, Random_Interferometer = True, export = True, tests = True):
    
    # Input initialization
    r_s, phi_s = input_state(r, m)
    A = input_matrix(r_s, phi_s, m, n)
    
    # Interferometer initialization
    if  Random_Interferometer == True:
        U, ind, phi, psi, eta = interferometer(n_bs, m)
    else:
        U = import_interferometer(path, "/matrix_U.dat")
        
    # The GBS device initializtion
    M = M_matrix(U, A)
    
    # Export all files related to the simulation
    if export == True:
        export_input(r_s, phi_s, A, ind, phi, psi, eta, N_BS, U, M, n)
        
    # Tests 
    if tests == True:
        print(f"Interferometer matrix U: {check_matrix(U)}; {check_unitarity(U)}")
        print(f"Gaussian matrix M: {check_matrix(M)} ; M.H * M {check_hermitianity(M.T.conj()@ M)}")
        assert len(M)==len(U)
 
    return M, U

def import_gbs_output(path):

    """Imports the output of the Gaussian Boson Sampling Emulator."""

    samples_data = np.genfromtxt(path + r"/samples.dat", dtype=str)
    samples_  = samples_data[:,0]
    samples = []
    
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    dir_alphabet = { alphabet[i]: 10+i for i in range(len(alphabet))}
    
    for s in samples_:
        sample = []
        for i in s:
            if i not in alphabet:
                sample.append(int(i))
            else:
                sample.append(dir_alphabet[str(i)])
                
        samples.append(sample)
    
    probs = []
    
    for i in range(len(samples_data)):
        probs.append(float(samples_data[i,1]))

    #test = np.genfromtxt(path + r"/test.dat", dtype=str)
    
    return np.array(samples), probs   #, test


# Optional 

def number_of_perm(n,m):
    
    if n > m: 
        return round(factorial(n)/factorial(n-m))  
    else:     
        return 0

def number_of_comb(n,m):
    
    if n > m:
        return round(factorial(n)/(factorial(n-m)*factorial(m)))
    else:
        return 0


In [167]:
# Tests 

def check_matrix(U):
    
    """
    Сhecks the matrix for zero elements.
    """
    
    condition = np.all(U)
    
    if condition == True:
        
        return 'None of the elements of the given matrix is zero'
    else:
        return 'There are zero elements of the given matrix'
    
def check_unitarity(U): 
    
    """
    Сhecks the matrix for unitarity.
    """
    
    condition = np.allclose(np.eye(len(U), dtype=np.complex128), U.T.conj() @ U)
    
    if condition == True:
        
        return 'The matrix is a unitary one'
    else:
        return 'The matrix is non-unitary one'
    
def check_hermitianity(M):
    
    """
    Сhecks the matrix for hermitianity.
    """
    
    condition = np.allclose(M, M.T.conj())
    
    if condition == True:
        
        return 'The matrix is a hermitian one'
    else:
        return 'The matrix is non-hermitian one'
    

## Initialize the Gaussian Boson Sampling Emulator

In [176]:
# Insert a relative path here (optional)

path = os.getcwd()
path = path.replace("\\" , "/" )

# Specify the emulation parameters: 

# Number of modes 
m = 5

# Number of input squeezed states
n = m 

# Squeezing parameter of the input squeezed vacuum states
r = 1.1

# Import the interferometer matrix (gen_U = False)
# or use a random unitary one (gen_U = True)
gen_U = False

# Number of beam splitters 
N_BS = m**2


In [183]:
M, U = device_initialization(m, n, r, Random_Interferometer = gen_U, export = False)

Interferometer matrix U: None of the elements of the given matrix is zero; The matrix is a unitary one
Gaussian matrix M: None of the elements of the given matrix is zero ; M.H * M The matrix is a hermitian one


## Run the Gaussian Boson Sampling Emulator

In [ ]:
# cmd = "python3 " path + r"GBS.py" 
# call(cmd.split(" "))

## Import data from the the Gaussian Boson Sampling Emulator

In [ ]:
samples, probabilities = import_gbs_output(path)